In [177]:
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score

In [178]:
insp = pd.read_csv("inspections.csv")
network = pd.read_csv("network.csv")
samplesub = pd.read_csv("sample_submission.csv")

In [179]:
samplesub = samplesub.merge(network, how="left", on="PipeId")

In [180]:
samplesub.isna().sum()

PipeId                 0
Incidence              0
Province               0
Town                   0
YearBuilt              0
Material               0
GasType                0
Diameter               0
Length                 0
Pressure               0
NumConnections         0
NumConnectionsUnder    0
BoolBridle             0
dtype: int64

In [181]:
insp = insp.merge(network, how ="inner", on = "PipeId")

In [182]:
insp.isna().sum()

PipeId                 0
MaintenanceId          0
InspectionYear         0
InspectionDate         0
MonthsLastRev          0
Severity               0
Incidence              0
Province               0
Town                   0
YearBuilt              0
Material               0
GasType                0
Diameter               0
Length                 0
Pressure               0
NumConnections         0
NumConnectionsUnder    0
BoolBridle             0
dtype: int64

In [183]:
insp = insp.drop("InspectionDate", axis=1)
insp = insp.drop("MaintenanceId", axis=1)

In [184]:
insp["AgeWhenInspected"] = insp["InspectionYear"] - insp["YearBuilt"]

In [185]:
insp_pure = insp.drop(columns=["Province","Town","YearBuilt","Material","GasType","Diameter","Length","Pressure","NumConnections","NumConnectionsUnder","BoolBridle"])

In [186]:
latest_year_index = insp.groupby('PipeId')['InspectionYear'].idxmax()
filtered_df = insp.loc[latest_year_index]

In [187]:
samplesub = samplesub.drop("Incidence",axis=1)

In [188]:
insp = insp.drop(columns=["PipeId","InspectionYear"],axis=1)

In [189]:
filtered_df = filtered_df[["PipeId","InspectionYear"]]

In [190]:
samplesub = samplesub.merge(filtered_df, how="left", on="PipeId")

In [191]:
samplesub = samplesub.rename(columns={"InspectionYear": "PipeLastInspectionYear"})

In [192]:
samplesub

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,PipeLastInspectionYear
0,446859944,Valencia,Alginet,2012,PE,Gas natural,110.0,44.286,0.150,0,0,False,2019.0
1,428124500,Valencia,Alginet,2012,PE,Gas natural,160.0,110.715,0.150,2,0,False,2019.0
2,438428871,Valencia,Alginet,2012,PE,Gas natural,110.0,13.575,0.150,0,0,False,2019.0
3,429034569,Valencia,Alginet,2012,PE,Gas natural,110.0,327.689,0.150,3,0,False,2019.0
4,411184477,Valencia,Alginet,2007,PE,Gas natural,200.0,932.762,0.150,0,0,False,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,Valencia,Paterna,2004,PE,Gas natural,110.0,7.656,0.025,0,0,False,2020.0
909729,235426673,Valencia,Paterna,2004,PE,Gas natural,110.0,4.383,0.025,2,0,False,2020.0
909730,235426707,Valencia,Paterna,2004,PE,Gas natural,110.0,2.292,0.025,1,0,False,2020.0
909731,190956601,Valencia,Paterna,2004,PE,Gas natural,90.0,87.809,4.000,0,0,False,2020.0


In [193]:
samplesub.isna().sum()

PipeId                        0
Province                      0
Town                          0
YearBuilt                     0
Material                      0
GasType                       0
Diameter                      0
Length                        0
Pressure                      0
NumConnections                0
NumConnectionsUnder           0
BoolBridle                    0
PipeLastInspectionYear    18685
dtype: int64

In [194]:
samplesub["AgeWhenInspected"] = samplesub["PipeLastInspectionYear"] - samplesub["YearBuilt"]


In [195]:
samplesub = samplesub.drop("PipeLastInspectionYear",axis=1)

In [196]:
samplesub.isna().sum()

PipeId                     0
Province                   0
Town                       0
YearBuilt                  0
Material                   0
GasType                    0
Diameter                   0
Length                     0
Pressure                   0
NumConnections             0
NumConnectionsUnder        0
BoolBridle                 0
AgeWhenInspected       18685
dtype: int64

In [197]:
samplesub['AgeWhenInspected'] = samplesub['AgeWhenInspected'].fillna(0)

In [198]:
samplesub.isna().sum()

PipeId                 0
Province               0
Town                   0
YearBuilt              0
Material               0
GasType                0
Diameter               0
Length                 0
Pressure               0
NumConnections         0
NumConnectionsUnder    0
BoolBridle             0
AgeWhenInspected       0
dtype: int64

In [199]:
insp = insp.drop(columns=["MonthsLastRev","Severity"],axis=1)

In [200]:
insp.isna().sum()

Incidence              0
Province               0
Town                   0
YearBuilt              0
Material               0
GasType                0
Diameter               0
Length                 0
Pressure               0
NumConnections         0
NumConnectionsUnder    0
BoolBridle             0
AgeWhenInspected       0
dtype: int64

In [201]:
insp

,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,AgeWhenInspected
0,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,9
1,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,11
2,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,13
3,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,15
4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,0,Valencia,Betera,1993,PE,Gas natural,63.0,1.237,4.000,0,0,False,27
6345340,0,Valencia,Betera,2000,PE,Gas natural,160.0,169.732,4.000,1,0,False,20
6345341,0,Barcelona,Sabadell,1995,FD,Gas natural,150.0,0.361,0.025,0,0,True,26
6345342,0,Valencia,Betera,1993,PE,Gas natural,63.0,23.306,4.000,0,0,False,27


In [202]:
data = insp

In [203]:
data_y = data["Incidence"]

In [204]:
data = data.drop(columns="Incidence",axis=1)

In [205]:
sub = samplesub

In [206]:
sub

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,AgeWhenInspected
0,446859944,Valencia,Alginet,2012,PE,Gas natural,110.0,44.286,0.150,0,0,False,7.0
1,428124500,Valencia,Alginet,2012,PE,Gas natural,160.0,110.715,0.150,2,0,False,7.0
2,438428871,Valencia,Alginet,2012,PE,Gas natural,110.0,13.575,0.150,0,0,False,7.0
3,429034569,Valencia,Alginet,2012,PE,Gas natural,110.0,327.689,0.150,3,0,False,7.0
4,411184477,Valencia,Alginet,2007,PE,Gas natural,200.0,932.762,0.150,0,0,False,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,Valencia,Paterna,2004,PE,Gas natural,110.0,7.656,0.025,0,0,False,16.0
909729,235426673,Valencia,Paterna,2004,PE,Gas natural,110.0,4.383,0.025,2,0,False,16.0
909730,235426707,Valencia,Paterna,2004,PE,Gas natural,110.0,2.292,0.025,1,0,False,16.0
909731,190956601,Valencia,Paterna,2004,PE,Gas natural,90.0,87.809,4.000,0,0,False,16.0


In [207]:
sub["Volume"] = ((3.14*(sub["Diameter"]/2000))**2*sub["Length"])
sub["VolumestimesPressure"] = sub["Volume"]*sub["Pressure"]
sub["VolumesdivPressure"] = sub["Volume"]/sub["Pressure"]
sub["Presstimesage"] = sub["Pressure"]*sub["AgeWhenInspected"]
sub["SurfaceArea"] = 2*3.1416*((sub["Diameter"]/2)**2) +  2*3.1416*((sub["Diameter"]/2)*sub["Length"])
sub["Circumference"] = 2*3.1416*(sub["Diameter"]/2)

sub["PresstimesSurf"] = sub["Pressure"] * sub["SurfaceArea"]
sub["PresstimesCircumference"] = sub["Pressure"] * sub["Circumference"] 
sub["AgesTimesLength"] = sub["AgeWhenInspected"] * sub["Length"]
sub["Connectionstimespressure"] = sub["NumConnections"] * sub["Pressure"]
sub["AgesTimesVol"] = sub["AgeWhenInspected"] * sub["Volume"]
sub["LenTimesDiameter"] = sub["Length"] * sub["Diameter"]

In [208]:
data["Volume"] = ((3.14*(data["Diameter"]/2000))**2*data["Length"])
data["VolumestimesPressure"] = data["Volume"]*data["Pressure"]
data["VolumesdivPressure"] = data["Volume"]/data["Pressure"]
data["Presstimesage"] = data["Pressure"]*data["AgeWhenInspected"]
data["SurfaceArea"] = 2*3.1416*((data["Diameter"]/2)**2) +  2*3.1416*((data["Diameter"]/2)*data["Length"])
data["Circumference"] = 2*3.1416*(data["Diameter"]/2)

data["PresstimesSurf"] = data["Pressure"] * data["SurfaceArea"]
data["PresstimesCircumference"] = data["Pressure"] * data["Circumference"] 
data["AgesTimesLength"] = data["AgeWhenInspected"] * data["Length"]
data["Connectionstimespressure"] = data["NumConnections"] * data["Pressure"]
data["AgesTimesVol"] = data["AgeWhenInspected"] * data["Volume"]
data["LenTimesDiameter"] = data["Length"] * data["Diameter"]

In [209]:
sub.to_csv("samplesubdefinitive.csv",index=False)
data.to_csv("datadefinitive.csv",index=False)
data_y.to_csv("data_ydefinitive.csv",index=False)